In [1]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableLambda
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langchain_core.messages import AIMessage, HumanMessage, BaseMessage
from langchain_core.chat_history import BaseChatMessageHistory, InMemoryChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

from langchain_community.llms import GPT4All
from langchain_community.embeddings import GPT4AllEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader

from langchain_classic.chains.history_aware_retriever import create_history_aware_retriever
from langchain_classic.chains.retrieval import create_retrieval_chain
from langchain_classic.chains.combine_documents import create_stuff_documents_chain

import os

c:\Users\Mekzedeck\AppData\Local\miniconda3\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
pdf_path = r"C:\Users\Mekzedeck\Desktop\ChatBot\nhif.pdf"
loader = PyPDFLoader(pdf_path)
documents = loader.load()

print(f"✅ Loaded {len(documents)} pages")

# Split into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=100)
texts = text_splitter.split_documents(documents)
print(f"✅ Split into {len(texts)} chunks")

✅ Loaded 31 pages
✅ Split into 72 chunks


In [3]:
embeddings = GPT4AllEmbeddings()

persist_directory = "./nhif_chroma_db"
vectorstore = Chroma.from_documents(
    texts, 
    embeddings,
    persist_directory=persist_directory
)

retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold", 
    search_kwargs={'score_threshold': 0.1, 'k': 4}
)

In [4]:
llm = GPT4All(
    model=r"C:\Users\Mekzedeck\AppData\Local\nomic.ai\GPT4All\Llama-3.2-3B-Instruct-Q4_0.gguf",
    n_threads=8
)

In [5]:
# Context-aware question rephraser (for chat history)
contextualize_q_prompt = ChatPromptTemplate.from_messages([
    ("system", """Given a chat history and the latest user question, 
    create a standalone question that incorporates context from the chat history.
    This helps retrieve relevant documents for the conversation."""),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}")
])

# Answer generation prompt
answer_prompt = ChatPromptTemplate.from_template("""
You are an NHIF expert assistant. Use the following context from NHIF documents to answer.

CONTEXT:
{context}

QUESTION: {input}

Guidelines:
- Answer ONLY based on the provided NHIF context
- If answer not in context, say: "I don't have that information in the NHIF documents."
- Be concise and accurate
- Reference NHIF policies/services when relevant

ANSWER:""")

# Create chains
history_aware_retriever = create_history_aware_retriever(
    llm, retriever, contextualize_q_prompt
)

question_answer_chain = create_stuff_documents_chain(llm, answer_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)


In [6]:

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    get_session_history,
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer",
)


In [7]:
def chat_with_nhif():
    print("\n" + "="*70)
    print("🤖 NHIF RAG CHATBOT - READY!")
    print("💡 Ask about NHIF services, coverage, claims, benefits, etc.")
    print("📝 Type 'quit', 'exit', or 'bye' to end")
    print("="*70)
    
    session_id = "nhif_session_001"
    
    while True:
        user_input = input("\nYou: ").strip()
        
        if user_input.lower() in ["quit", "exit", "bye"]:
            print("\n👋 Thank you for using NHIF Chatbot!")
            break
        
        if not user_input:
            print("Please enter a question about NHIF.")
            continue
        
        try:
            print("🤔 Searching NHIF documents...")
            result = conversational_rag_chain.invoke(
                {"input": user_input},
                config={"configurable": {"session_id": session_id}}
            )
            
            response = result["answer"]
            print(f"\n✅ Assistant: {response}")
            
        except Exception as e:
            print(f"❌ Error: {str(e)}")
            print("Please try rephrasing your question.")

In [8]:
if __name__ == "__main__":
    try:
        chat_with_nhif()
        
        # Show final chat history
        print("\n📜 Final Chat History:")
        if "nhif_session_001" in store:
            print("-" * 50)
            for i, message in enumerate(store["nhif_session_001"].messages):
                prefix = "🤖 AI" if isinstance(message, AIMessage) else "👤 You"
                print(f"{prefix}: {message.content[:100]}...")
            print("-" * 50)
        
    except KeyboardInterrupt:
        print("\n👋 Chat interrupted.")
    except Exception as e:
        print(f"❌ Unexpected error: {str(e)}")


🤖 NHIF RAG CHATBOT - READY!
💡 Ask about NHIF services, coverage, claims, benefits, etc.
📝 Type 'quit', 'exit', or 'bye' to end
🤔 Searching NHIF documents...

✅ Assistant:  
JE NHIF NI ZANZIBAR HEALTH SERVICES FUND (ZHSF) NA UHC. 

EXPLANATION:
NHIF ni kifungo cha afya ambacho kinatoa huduma za afya kwa wafanyakazi na wanajeshi wa Tanzania, pamoja na watu walio katika hali ya kutokuwa na rasilimali (vijana). NHIF inatangaza UHC ili kuwawezesha watu binafsi kupata huduma za afya kila wakati. ZHSF ni mfungo wa afya ambao unatoa huduma za afya katika Zanzibar, na uhusiano wake na NHIF ni muhimu kwa UHC.

Kumbuka: Jibu hili linatokana na maelezo ya NHIF juu ya UHC na ZHSF. 

Jibu la mwisho ni JE NHIF NI ZANZIBAR HEALTH SERVICES FUND (ZHSF) NA UHC. 
Jibu la mwisho ni: JE NHIF NI ZANZIBAR HEALTH SERVICES FUND (ZHSF) NA UHC.

👋 Thank you for using NHIF Chatbot!

📜 Final Chat History:
--------------------------------------------------
👤 You: JE NHIF NI NINI?...
🤖 AI:  
JE NHIF NI ZANZIBAR HEAL